In [1]:
!nvidia-smi

Tue Jul 11 18:32:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install git+https://github.com/bofenghuang/vigogne

  Cloning https://github.com/bofenghuang/vigogne to /tmp/pip-req-build-aafrrobr
  Running command git clone --filter=blob:none --quiet https://github.com/bofenghuang/vigogne /tmp/pip-req-build-aafrrobr
  Resolved https://github.com/bofenghuang/vigogne to commit e44d50c690f2f760a58f6c1f66cf77162b9161ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-etcv0brj/accelerate_1d85c2b1b6d045529f1213bda7a6e5f9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-etcv0brj/accelerate_1d85c2b1b6d045529f1213bda7a6e5f9
  Resolved https://github.com/huggingface/accelerate.git to commit 7954a28a71d484c4182a6b1074c1b8cc51642fc9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

In [3]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from vigogne.preprocess import generate_instruct_prompt


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [5]:
base_model_name_or_path = "bofenghuang/vigogne-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, padding_side="right", use_fast=False)
# tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    # trust_remote_code=True,
    low_cpu_mem_usage=True,
)

# lora_model_name_or_path = ""
# model = PeftModel.from_pretrained(model, lora_model_name_or_path)

model.eval()

if torch.__version__ >= "2":
    model = torch.compile(model)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
def instruct(
    user_query ,
    temperature=0.1,
    max_new_tokens=512,
    **kwargs,
):
    prompt = generate_instruct_prompt(user_query)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            do_sample=temperature > 0.0,
            max_new_tokens=max_new_tokens,
            **kwargs,
          ),
        return_dict_in_generate=True,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text

In [11]:
instruct("Explique le théorème central limite.")

'Le théorème central limite stipule que la moyenne des valeurs de variables aléatoires est égale à leur moyenne arithmétique, quelle que soit la distribution de probabilité sous-jacente. Cela signifie que la forme de la distribution de probabilité de la moyenne des valeurs de variables aléatoires est la même que celle de la distribution de probabilité de la moyenne arithmétique de ces variables.'

In [8]:
instruct("Répondez à la question suivante : Les pratiques artistiques transforment-elles le monde ?")

'Oui, les pratiques artistiques peuvent transformer le monde en créant des expériences visuelles et émotionnelles qui peuvent être utilisées pour exprimer des idées et des sentiments. Les artistes peuvent également utiliser leurs créations pour sensibiliser les gens à des problèmes sociaux et politiques, ou pour promouvoir des idées positives.'

In [12]:
instruct("Traduisez le texte suivant en français : Caterpillars extract nutrients which are then converted into butterflies. People have extracted billions of nuggets of understanding and GPT-4 is humanity's butterfly.")

"Les papillons de nuit extraient des nutriments qui sont ensuite convertis en papillons. Les gens ont extrait des milliards de nuggets d'apprentissage et GPT-4 est l'oiseau de proie de l'humanité."

In [13]:
print(instruct("Écrivez une fonction qui prend une liste de chaînes de caractères et renvoie une liste sans doublons."))

Voici une fonction qui prend une liste de chaînes de caractères et renvoie une liste sans doublons :

def remove_duplicates(strings):
    result = []
    for s in strings:
        if s not in result:
            result.append(s)
    return result

La fonction prend une liste de chaînes de caractères et renvoie une liste sans doublons en ajoutant chaque chaîne de caractères à une liste de chaînes de caractères si elle n'est pas déjà présente dans la liste. Si la chaîne de caractères est déjà présente dans la liste, elle est supprimée.
